<a href="https://colab.research.google.com/github/vamshap/PySpark-Challenges/blob/main/StockPricesUp_Down.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# Import required types
from pyspark.sql.types import StructType, StructField, StringType, DateType, FloatType
from pyspark.sql import SparkSession
from datetime import datetime as date_time
from pyspark.sql.window import Window
from pyspark.sql.functions import lag,col,when
# Create Spark session
spark = SparkSession.builder.appName("StockData").getOrCreate()

# Define the schema
stock_schema = StructType([
    StructField("Stock_id", StringType(), False),
    StructField("date", DateType(), False),
    StructField("price", FloatType(), False)
])
date_time.strptime('07/10/2022 09:00:00', '%m/%d/%Y %H:%M:%S')
# Create the data
stock_data = [
    ("A", date_time.strptime('2024-01-01','%Y-%m-%d'), 100.0),
    ("A", date_time.strptime('2024-01-02','%Y-%m-%d'), 105.0),
    ("A", date_time.strptime('2024-01-03','%Y-%m-%d'), 104.0),
    ("B", date_time.strptime('2024-01-01','%Y-%m-%d'), 200.0),
    ("B", date_time.strptime('2024-01-02','%Y-%m-%d'), 200.0),
    ("B", date_time.strptime('2024-01-03','%Y-%m-%d'), 201.0)
]

# Create DataFrame
stock_df = spark.createDataFrame(data=stock_data, schema=stock_schema)

# If you want to create a temporary view to query using SQL
stock_df.show()
WindowSpec = Window().partitionBy("Stock_id").orderBy("date")

stock_df = stock_df.withColumn("PreviousStock",lag("price").over(WindowSpec))
stock_df.show()
level_price = stock_df.withColumn("status",(when(col("price") > col("PreviousStock"), "down" )
                             .when(col("price") < col("PreviousStock") , "Up" ).otherwise ("Base Price") ))

level_price.show()



+--------+----------+-----+
|Stock_id|      date|price|
+--------+----------+-----+
|       A|2024-01-01|100.0|
|       A|2024-01-02|105.0|
|       A|2024-01-03|104.0|
|       B|2024-01-01|200.0|
|       B|2024-01-02|200.0|
|       B|2024-01-03|201.0|
+--------+----------+-----+

+--------+----------+-----+-------------+
|Stock_id|      date|price|PreviousStock|
+--------+----------+-----+-------------+
|       A|2024-01-01|100.0|         NULL|
|       A|2024-01-02|105.0|        100.0|
|       A|2024-01-03|104.0|        105.0|
|       B|2024-01-01|200.0|         NULL|
|       B|2024-01-02|200.0|        200.0|
|       B|2024-01-03|201.0|        200.0|
+--------+----------+-----+-------------+

+--------+----------+-----+-------------+----------+
|Stock_id|      date|price|PreviousStock|    status|
+--------+----------+-----+-------------+----------+
|       A|2024-01-01|100.0|         NULL|Base Price|
|       A|2024-01-02|105.0|        100.0|      down|
|       A|2024-01-03|104.0|      